In [1]:
import os
import json
import warnings
import numpy as np

In [2]:
origen_libs = '/home/scopatz/origen22/libs'
tape9 = 'decay.lib'
lib = os.path.join(origen_libs, tape9)

In [3]:
from pyne import utils
utils.toggle_warnings()
warnings.simplefilter('ignore')
from pyne.origen22 import parse_tape9
from pyne import nucname

In [4]:
toname = lambda nuc: nucname.name(nucname.zzaaam_to_id(int(nuc)))
ln2 = np.log(2.0)

In [5]:
t9 = parse_tape9(lib)

In [6]:
nlb = tuple(sorted(t9.keys()))

In [7]:
nlb

(1, 2, 3)

In [8]:
t9[3]['half_life']

{10030: 389700000.0,
 30060: inf,
 30070: inf,
 40090: inf,
 40100: 50490000000000.0,
 60140: 180800000000.0,
 270720: 0.1227,
 270730: 0.11550000000000001,
 270740: 0.1075,
 270750: 0.080159999999999995,
 280660: 196600.0,
 280720: 2.419,
 280730: 0.39350000000000002,
 280740: 0.64829999999999999,
 280750: 0.17960000000000001,
 280760: 0.26840000000000003,
 280770: 0.1028,
 280780: 0.1376,
 290660: 306.0,
 290670: 222700.0,
 290720: 6.0019999999999998,
 290730: 3.948,
 290740: 0.57310000000000005,
 290750: 0.76659999999999995,
 290760: 0.22109999999999999,
 290770: 0.29459999999999997,
 290780: 0.1206,
 290790: 0.1474,
 290800: 0.0911,
 290810: 0.074469999999999995,
 300660: inf,
 300670: inf,
 300680: inf,
 300690: 3420.0,
 300691: 49540.0,
 300700: inf,
 300710: 144.0,
 300711: 14112.0,
 300720: 167400.0,
 300730: 23.5,
 300740: 95.0,
 300750: 9.0,
 300760: 5.4000000000000004,
 300770: 1.3999999999999999,
 300780: 2.4289999999999998,
 300790: 0.3821,
 300800: 0.71130000000000004,
 3

In [9]:
DECAY_RXS = ['bminus', 'bplus', 'ec', 'alpha', 'it', 'sf', 'bminus_n']
paroff = {'frac_beta_minus': 10000000, 'frac_beta_minus_x': 10000001, 
  'frac_beta_plus_or_electron_capture': -10000000,
  'frac_beta_plus_or_electron_capture_x': -9999999,
  'frac_alpha': -20040000,
  'frac_isomeric_transition': -1,
  #'frac_spont_fiss': 0,
  'frac_beta_n': 9990000}
paroffm = {'frac_beta_minus': 9999999, 'frac_beta_minus_x': 10000000, 
  'frac_beta_plus_or_electron_capture': -10000001,
  'frac_beta_plus_or_electron_capture_x': -10000000,
  'frac_alpha': -20040001,
  'frac_isomeric_transition': -1,
  #'frac_spont_fiss': 0,
  'frac_beta_n': 9989999}
lambdas = {}
gamma_maps = {}
for n in nlb:
    lambdas.update({'lambda_' + toname(nuc): ln2/val for nuc, val in t9[n]['half_life'].items() if nuc != 162500})
    for key in t9[n]:
        if not key.startswith('frac_') or key == 'frac_natural_abund' or key == 'frac_spont_fiss':
            continue
        for nuc, val in t9[n][key].items():
            if val < 1e-16:
                continue  # forbidden decay
            nname = toname(nuc)
            if lambdas['lambda_' + nname] < 1e-16:
                continue  # stable nuclide
            if nname not in gamma_maps:
                gamma_maps[nname] = {}
            poff = paroff if int(nuc)%10 == 0 else paroffm
            child = nucname.zzaaam_to_id(int(nuc)) + poff[key]
            cname = nucname.name(child)
            gname = 'gamma_{0}_{1}_{2}'.format(nname, cname, key[5:])
            gamma_maps[nname][gname] = val
            
# add β- decays
for key, val in lambdas.items():
    if val < 1e-16:
        continue  # stable nuclide
    _, _, nname = key.rpartition('_')
    if nname not in gamma_maps:
        gamma_maps[nname] = {}
    gamma_total = sum(gamma_maps[nname].values())
    if gamma_total < 1.0:
        parid = nucname.id(nname)
        poff = paroff if parid%10 == 0 else paroffm
        child = parid + poff['frac_beta_minus']
        cname = nucname.name(child)
        gname = 'gamma_{0}_{1}_{2}'.format(nname, cname, 'beta_minus')
        gamma_maps[nname][gname] = 1.0 - gamma_total

In [10]:
symbols = dict(lambdas)
for val in gamma_maps.values():
    symbols.update(val)

In [11]:
with open('transmute_data.json', 'r') as f:
    data = json.load(f)
data['symbols'] = symbols
with open('transmute_data.json', 'w') as f:
    json.dump(data, f, indent='    ', sort_keys=True)